In [ ]:
import csv 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

# Create and open the CSV file for writing
with open('150 number_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    # Write the header row
    writer.writerow(['Name', 'Age', 'Location', 'Addresses', 'Number 01', 'Number 02'])
    
    # Open the input CSV file to read phone numbers
    with open('150 number - Sheet1.csv', 'r', newline='') as inputfile:
        reader = csv.reader(inputfile)
        next(reader)  # Skip the header row
        
        for row in reader:
            driver = webdriver.Chrome()
            driver.get("https://www.411.com/reverse-phone-lookup")
            
            # Extract phone number from the row
            phone_number = row[0]
            
            try:
                # Find the input field and input the phone number
                phone_input = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, "//input[@name='s_phone']"))
                )
                phone_input.clear()
                phone_input.send_keys(phone_number)
                
                # Click on the submit button
                submit_btn = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']"))
                )
                submit_btn.click()
                
                # Handle checkbox and TOS button (if present)
                try:
                    check_box = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, "//input[@type='checkbox']"))
                    )
                    check_box.click()
                    time.sleep(3)
                except Exception as e:
                    print(f"An error occurred while finding or clicking the checkbox: {e}")
                
                try:
                    check_btn = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[@id='tos-button']"))
                    )
                    check_btn.click()
                    time.sleep(3)
                except Exception as e:
                    print(f"An error occurred: {e}")
                
                # Wait for the page to load
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@id='tos-overlay']")))
                
                # Extract information from the page
                try:
                    name = driver.find_element(By.XPATH, "//*[@id=\"owner-name\"]/span[2]").text
                except Exception as e:
                    name = "blank area"
                    print("Error retrieving first name:", e)
                
                try:
                    age = driver.find_element(By.XPATH, "//*[@id=\"owner-age\"]/span[2]").text
                except Exception as e:
                    age = "blank area"
                    print("Error retrieving age:", e)
                
                try:
                    location = driver.find_element(By.XPATH, "//*[@id=\"owner-location\"]/span[2]").text
                except Exception as e:
                    location = "blank area"
                    print("Error retrieving location:", e)
                
                # Scroll the window
                try:
                    driver.execute_script("window.scrollTo(0, 200);")
                    time.sleep(2)
                except Exception as e:
                    print(f"An error occurred while scrolling the window: {e}")
                
                # Extract owner's addresses
                try:
                    owner_addresses = driver.find_element(By.XPATH, "//div[@id='other-locations-0']").text
                except Exception as e:
                    owner_addresses = "blank area"
                    print("Error retrieving owner's addresses:", e)
                
                # Scroll further down
                try:
                    driver.execute_script("window.scrollTo(300, 700);")
                    time.sleep(2)
                except Exception as e:
                    print(f"An error occurred while scrolling the window: {e}")
                
                # Extract owner's landline numbers
                try:
                    owner_landlines_number_1 = driver.find_element(By.XPATH, "//*[@id=\"landline-0\"]").text
                except Exception as e:
                    owner_landlines_number_1 = "blank area"
                    print("Error retrieving owner's landline number 1:", e)
                
                try:
                    owner_landlines_number_2 = driver.find_element(By.XPATH, "//*[@id=\"landline-1\"]").text
                except Exception as e:
                    owner_landlines_number_2 = "blank area"
                    print("Error retrieving owner's landline number 2:", e)
                
                # Write the extracted data to the CSV file
                writer.writerow([name, age, location, owner_addresses, owner_landlines_number_1, owner_landlines_number_2])
            
            except Exception as e:
                print(f"An error occurred during processing: {e}")
            
            finally:
                driver.quit()